In [1]:
!pip install pytorch_pretrained_bert
!pip3 install fairseq
!pip3 install fastbpe
!pip3 install vncorenlp
!pip3 install transformers
!pip3 install fairseq
!pip3 install fastBPE
!pip install underthesea

     |████████████████████████████████| 133kB 5.6MB/s 
     |████████████████████████████████| 133kB 15.8MB/s 
     |████████████████████████████████| 6.8MB 21.9MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
ERROR: botocore 1.19.22 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 686kB 5.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 122kB 24.6MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 112kB 30.3MB/s 
     |████████████████████████████████| 276kB 33.3MB/s 
  Created wheel for fairseq: filename=fairseq-0.10.0-cp36-cp36m-linux_x86_64.whl size=2631901 sha256=af21812f7159b08a5af5e82920b566998bbeabaf6d35181fe109bbf840ddd299
  Stored in directory: 

In [2]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2020-11-20 17:09:00--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 13.225.103.51, 13.225.103.117, 13.225.103.113, ...
Connecting to public.vinai.io (public.vinai.io)|13.225.103.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz’

PhoBERT_base_fairse 100%[===================>]   1.16G  91.0MB/s    in 13s     

2020-11-20 17:09:15 (90.1 MB/s) - ‘PhoBERT_base_fairseq.tar.gz’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


In [3]:
!ls PhoBERT_base_fairseq

bpe.codes  dict.txt  model.pt


In [4]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from nltk.tokenize import sent_tokenize
import torch 
from pytorch_pretrained_bert import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np

In [6]:
from fairseq.models.roberta import RobertaModel
phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT.eval()  # disable dropout (or leave in train mode to finetune

2020-11-20 17:09:46 | INFO | fairseq.file_utils | loading archive file PhoBERT_base_fairseq
2020-11-20 17:09:47 | INFO | fairseq.tasks.masked_lm | dictionary: 64000 types
2020-11-20 17:09:53 | INFO | fairseq.file_utils | https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json not found in cache, downloading to /tmp/tmpoi658fxi
1042301B [00:00, 1243835.49B/s]
2020-11-20 17:09:55 | INFO | fairseq.file_utils | copying /tmp/tmpoi658fxi to cache at /root/.cache/torch/pytorch_fairseq/e2aab4d600e7568c2d88fc7732130ccc815ea84ec63906cb0913c7a3a4906a2e.0f323dfaed92d080380e63f0291d0f31adfa8c61a62cbcb3cb8114f061be27f7
2020-11-20 17:09:55 | INFO | fairseq.file_utils | creating metadata file for /root/.cache/torch/pytorch_fairseq/e2aab4d600e7568c2d88fc7732130ccc815ea84ec63906cb0913c7a3a4906a2e.0f323dfaed92d080380e63f0291d0f31adfa8c61a62cbcb3cb8114f061be27f7
2020-11-20 17:09:55 | INFO | fairseq.file_utils | removing temp file /tmp/tmpoi658fxi
2020-11-20 17:09:55 | INFO | fairseq.file_utils | http

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(64001, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(258, 768, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=T

In [7]:
from fairseq.data.encoders.fastbpe import fastBPE

# Khởi tạo Byte Pair Encoding cho PhoBERT
class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

In [8]:
args = BPE()
phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

In [28]:
def get_representation(model, tokenized_sentences):
  tokenized_ids = [model.encode(sent) for sent in tokenized_sentences]
  tokenized_tensor = [torch.tensor(item) for item in tokenized_ids]
  encoded_list = []
  for i in range(len(tokenized_tensor)):
    with torch.no_grad():
      encoded_layers = model.extract_features(tokenized_tensor[i])
    encoded_list.append(encoded_layers)
  sentence_list = [torch.mean(vec, dim=1).reshape((768)).numpy() for vec in encoded_list]
  return sentence_list

In [29]:
from sklearn.metrics import pairwise_distances_argmin_min
def get_index_from_kmeans(sentence_embedding_list):
    """
    Input a list of embeded sentence vectors
    Output an list of indices of sentence in the paragraph, represent the clustering of key sentences
    Note: Kmeans is used here for clustering
    """
    n_clusters = np.ceil(len(sentence_embedding_list)**0.5)
    kmeans = KMeans(n_clusters=int(n_clusters))
    kmeans = kmeans.fit(sentence_embedding_list)
    sum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric='euclidean')
    sum_index = sorted(sum_index)
    return sum_index

In [33]:
def get_summarization(text):
  tokenized_sentences = sent_tokenize(text)
  sentences_representation = get_representation(phoBERT, tokenized_sentences)
  sum_index = get_index_from_kmeans(sentences_representation)
  summary = ' '.join([tokenized_sentences[ind] for ind in sum_index])
  return summary

In [34]:
text = '''
Được phát triển dựa trên công nghệ AI nổi tiếng là Generative Adversarial Networks, phần mềm miễn phí này hoạt động nhờ hai mạng nơ-ron. Trong đó, mạng nơ-ron thứ hai có nhiệm vụ chỉ dẫn chi nơ-ron đầu tiên tái tạo màu của hình ảnh tốt hơn. Đây là một trong những công cụ chỉnh màu tốt nhất được các chuyên gia đánh giá. Nó cho ra kết quả màu sắc chân thực từ con người tới ngoại cảnh. Tuy nhiên

DeOldify chạy trên Ubuntu, một bản phân phối Linux phổ biến. Do đó, để mang phần mềm này về máy tính của mình, người dùng sẽ cần có một số kiến thức công nghệ phù hợp để thực hiện được các thao tác.
'''

In [35]:
print(get_summarization(text))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until



Được phát triển dựa trên công nghệ AI nổi tiếng là Generative Adversarial Networks, phần mềm miễn phí này hoạt động nhờ hai mạng nơ-ron. Trong đó, mạng nơ-ron thứ hai có nhiệm vụ chỉ dẫn chi nơ-ron đầu tiên tái tạo màu của hình ảnh tốt hơn. Đây là một trong những công cụ chỉnh màu tốt nhất được các chuyên gia đánh giá.


In [43]:
text = '''Theo báo Sài_Gòn Giải_Phóng , ngày 17/3 , Công_an quận Gò_Vấp , TPHCM đang điều_tra truy_xét vụ trộm xe ô_tô hiệu Ford_Ranger ở chung_cư Hà_Đô , phường 3 , quận Gò_Vấp .
Thông_tin trên báo Thanh_Tra , Trung_tá Nguyễn_Đăng_Sơn , Phó Đội_trưởng đội Tổng_hợp , Công_an quận Gò_Vấp cho biết , theo thông_tin ban_đầu , vào_khoảng 10h ngày 13/3/2019 , anh Mai_Minh_N. ( sinh năm 1975 , tạm_trú tại lô D chung_cư Hà_Đô ) khi xuống lấy xe tại bãi giữ xe chung_cư Hà_Đô thì phát_hiện chiếc ô_tô hiệu Ford_Ranger , biển kiểm_soát : 51C-980.xx bị mất .
Vụ_việc nhanh_chóng được trình_báo đến Công_an địa_phương .
Nhận tin báo , Công_an quận Gò_Vấp có_mặt khám_nghiệm hiện_trường , lấy lời khai , điều_tra truy_xét .
Qua trích xuất camera an_ninh , công_an xác_định lúc 21h26 ngày 12/3 , một thanh_niên ( chưa rõ lai_lịch ) đi_lại chiếc xe của anh N. gửi ở bãi xe .
Sau đó , thanh_niên này mở_cửa , nổ máy chiếc xe ô_tô trên và rời khỏi bãi giữ xe .
Hình_ảnh ghi lại đối_tượng thực_hiện vụ trộm_cắp khá mờ nên việc điều_tra truy_xét khó_khăn .
Báo Tri_Thức_Trẻ thông_tin thêm , anh N. cho biết , chiếc ô_tô bị mất anh mua cách đây khoảng một năm , trị_giá gần 1 tỷ đồng , xe sắp hết hạn bảo_hiểm .
Sáng ngày 13/3/2019 , khi anh xuống lấy xe để đi mua bảo_hiểm thì phát_hiện chiếc xe không còn ở bãi giữ xe .
Hiện vụ_việc đang được điều_tra làm rõ .
H.Y ( tổng_hợp )'''

gold = '''Xuống nhà_xe chung_cư Hà_Đô lấy xe , một người_dân bàng_hoàng phát_hiện chiếc xế hộp tiền tỷ của mình sau một đêm đã không_cánh_mà_bay .'''

In [44]:
summ = get_summarization(text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
print(summ)

Theo báo Sài_Gòn Giải_Phóng , ngày 17/3 , Công_an quận Gò_Vấp , TPHCM đang điều_tra truy_xét vụ trộm xe ô_tô hiệu Ford_Ranger ở chung_cư Hà_Đô , phường 3 , quận Gò_Vấp . Qua trích xuất camera an_ninh , công_an xác_định lúc 21h26 ngày 12/3 , một thanh_niên ( chưa rõ lai_lịch ) đi_lại chiếc xe của anh N. gửi ở bãi xe . Hiện vụ_việc đang được điều_tra làm rõ . H.Y ( tổng_hợp )


In [41]:
!pip install rouge

In [42]:
from rouge import Rouge
rouge = Rouge()

In [46]:
rouge.get_scores(summ, gold)

[{'rouge-1': {'f': 0.14583332968967022,
   'p': 0.0958904109589041,
   'r': 0.30434782608695654},
  'rouge-2': {'f': 0.042553187904029274,
   'p': 0.027777777777777776,
   'r': 0.09090909090909091},
  'rouge-l': {'f': 0.17948717543721246, 'p': 0.125, 'r': 0.3181818181818182}}]